In [1]:
import sys
import os
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score
from gensim import models
from scipy import sparse
from scipy.sparse import csr_matrix, coo_matrix 
path_build = "../../data/tencent2020/build/"
path_save = "../../data/tencent2020/save/"
pickle_path = "../../data/tencent2020/pickle"
sub_path = "../../data/tencent2020/sub"

train_preliminary_p = path_build + "train_preliminary/"

if not os.path.exists(pickle_path):
    os.mkdir(pickle_path) 
if not os.path.exists(sub_path):
    os.mkdir(sub_path)

In [2]:
user_ids = pd.read_pickle(f"{pickle_path}/user_ids_relencode.pickle")
user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
user_ids = user_ids.merge(user, how='left', on='user_id')
user_ids.head()

,user_id,creative_id,ad_id,product_id,product_category,advertiser_id,industry,click_times,time,creative_id_times,...,max_clicktimes,min_clicktimes,mean_time,max_time,min_time,time_len,time_clicktimes,time_creativeids,age,gender
0,1,"[642275, 170988, 685687, 1307085, 100774, 5953...","[581171, 156554, 620126, 1166887, 92441, 55685...","[1, 125, 1, 1, 1132, 1, 1, 1, 1, 1244, 55, 55,...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[6411, 8522, 26781, 12996, 10113, 13007, 15341...","[323, 7, 104, 323, 1, 323, 73, 214, 64, 235, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]","[20, 20, 20, 39, 40, 43, 46, 52, 60, 64, 64, 7...","[3872964, 1123872, 3935014, 706993, 226871, 37...",...,2,1,47.461538,76,20,10,"[3, 1, 1, 1, 1, 1, 1, 2, 1, 2]","[3, 1, 1, 1, 1, 1, 1, 2, 1, 1]",4.0,1.0
1,2,"[52634, 128035, 32734, 478744, 10372, 349730, ...","[49388, 117016, 31581, 435462, 11432, 318236, ...","[76, 69, 118, 118, 1193, 76, 1, 1064, 1, 1, 10...","[2, 2, 2, 2, 2, 2, 18, 2, 18, 18, 2, 18, 2, 18...","[20681, 9438, 16619, 23545, 705, 20681, 31504,...","[315, 235, 7, 7, 314, 315, 47, 7, 47, 47, 239,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10, 11, 14, 17, 28, 28, 28, 38, 38, 39, 41, 4...","[3670932, 575824, 3013599, 3644043, 316325, 34...",...,2,1,53.822222,90,10,28,"[1, 1, 1, 1, 3, 2, 1, 1, 3, 2, 1, 4, 1, 4, 1, ...","[1, 1, 1, 1, 3, 2, 1, 1, 3, 2, 1, 4, 1, 4, 1, ...",10.0,1.0
2,3,"[518448, 632295, 556265, 645498, 466848, 56861...","[470940, 572190, 504577, 584182, 424822, 51556...","[31600, 35900, 1444, 31352, 1444, 1, 1, 857, 1...","[17, 17, 2, 17, 2, 18, 18, 2, 2, 2, 18, 12, 2,...","[30056, 8682, 16554, 12593, 15183, 7955, 7362,...","[1, 1, 319, 1, 319, 7, 54, 7, 319, 319, 202, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12, 13, 14, 14, 14, 17, 19, 22, 31, 36, 37, 4...","[3698944, 3858546, 3751294, 3877415, 3627528, ...",...,1,1,50.266667,88,12,23,"[1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, ...","[1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, ...",7.0,2.0
3,4,"[32621, 464022, 452725, 1466785, 1520025, 1750...","[31491, 422207, 412076, 1308931, 1355934, 1558...","[1620, 1, 2363, 33897, 36230, 32982, 34972, 34...","[2, 18, 2, 17, 17, 4, 17, 17, 18, 2, 2, 2, 2, ...","[17456, 7015, 11589, 10744, 21413, 745, 8954, ...","[235, 26, 245, 1, 1, 1, 1, 1, 86, 316, 235, 7,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[8, 15, 41, 44, 48, 48, 48, 48, 49, 52, 58, 58...","[3000797, 3623682, 3608553, 918527, 988666, 12...",...,1,1,57.724138,84,8,15,"[1, 1, 1, 1, 4, 1, 1, 2, 1, 1, 6, 1, 2, 2, 4]","[1, 1, 1, 1, 4, 1, 1, 2, 1, 1, 6, 1, 2, 2, 4]",5.0,1.0
4,5,"[238161, 280415, 19832, 35721, 666206, 821172,...","[218107, 256798, 20328, 34244, 602347, 740737,...","[1, 1, 76, 125, 1, 125, 1, 1, 1, 889, 1, 2358,...","[5, 8, 2, 2, 18, 2, 5, 18, 18, 2, 18, 2, 5, 5,...","[10525, 923, 20681, 8526, 35446, 2611, 15865, ...","[294, 1, 315, 7, 319, 7, 285, 319, 316, 235, 3...","[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 13, 14, 15, 20, 21, 24, 25, 27, 28, 29, 30...","[1996086, 2547847, 1466359, 3357252, 3907306, ...",...,2,1,44.424242,87,3,26,"[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, ...",4.0,1.0


In [3]:
w2v_features = [
#     {'name':'creative_id', 'size':128, 'windows':5, 'min_count':1, 'version':2},
#     {'name':'ad_id', 'size':128, 'windows':5, 'min_count':1, 'version':2},
#     {'name':'advertiser_id', 'size':64, 'windows':5, 'min_count':1, 'version':2},
#     {'name':'product_id', 'size':64, 'windows':5, 'min_count':1, 'version':2},
#     {'name':'industry', 'size':32, 'windows':5, 'min_count':1, 'version':2},
#     {'name':'product_category', 'size':16, 'windows':5, 'min_count':1, 'version':2},
#     {'name':'time', 'size':16, 'windows':5, 'min_count':1, 'version':2},
#     {'name':'click_times', 'size':8, 'windows':5, 'min_count':1, 'version':2},
    
#     {'name':'creative_id_times', 'size':128, 'windows':5, 'min_count':1, 'version':2},
#     ##{'name':'ad_id_times', 'size':128, 'windows':5, 'min_count':1, 'version':2},
#     {'name':'product_id_times', 'size':64, 'windows':5, 'min_count':1, 'version':2},
#     {'name':'advertiser_id_times', 'size':64, 'windows':5, 'min_count':1, 'version':2},
#     {'name':'product_category_times', 'size':32, 'windows':5, 'min_count':1, 'version':2},
#     {'name':'industry_times', 'size':32, 'windows':5, 'min_count':1, 'version':2},
    
##     {'name':'creative_id_t', 'size':128, 'windows':5, 'min_count':1, 'version':2},
##     {'name':'ad_id_t', 'size':128, 'windows':5, 'min_count':1, 'version':2},
##     {'name':'product_id_t', 'size':64, 'windows':5, 'min_count':1, 'version':2},
##     {'name':'advertiser_id_t', 'size':64, 'windows':5, 'min_count':1, 'version':2},
##     {'name':'product_category_t', 'size':32, 'windows':5, 'min_count':1, 'version':2},
##     {'name':'industry_t', 'size':32, 'windows':5, 'min_count':1, 'version':2},
    
    {'name':'time_clicktimes', 'size':91, 'windows':10, 'min_count':1, 'version':2,},
    {'name':'time_creativeids', 'size':91, 'windows':10, 'min_count':1, 'version':2 },
    
]
for fea_dict in w2v_features:
    print(fea_dict)
    name = fea_dict['name']
    size = fea_dict['size']
    windows = fea_dict['windows']
    min_count = fea_dict['min_count']
    version = fea_dict['version']
    user_ids[name] = user_ids[name].map(lambda x: [str(i) for i in x])
    w2v = models.Word2Vec(user_ids[name].values, sg=1, size=size, window=windows, workers=6, hs=1, min_count=min_count)
    w2v.wv.save_word2vec_format(f"{path_save}/{name}_w2v_{windows}_{version}.bin",binary=True)
    models.Word2Vec()
    #保存矩阵
    #w2v = models.KeyedVectors.load_word2vec_format(f"{path_save}/{name}_w2v_{size}.bin",binary=True)
    w2v = models.KeyedVectors.load_word2vec_format(f"{path_save}/{name}_w2v_{windows}_{version}.bin",binary=True)
    if 'vocab_size' in fea_dict:
        w2v_vocab_size = fea_dict['vocab_size']
    else:
        w2v_vocab_size = len(w2v.vocab) + 1
        print(w2v_vocab_size)
    w2v_matrix = np.zeros((w2v_vocab_size, w2v.vector_size))
    for w in w2v.vocab:
        w2v_matrix[int(w)] = w2v[w]
    save_pickle(w2v_matrix.astype(np.float16), f"{path_save}/{name}_w2v_matrix2.pkl",)

{'name': 'time_clicktimes', 'size': 91, 'windows': 10, 'min_count': 1, 'version': 2}
441
{'name': 'time_creativeids', 'size': 91, 'windows': 10, 'min_count': 1, 'version': 2}
397


Word2Vec参数：  
hs ({0, 1}, optional) – 1: 采用hierarchical softmax训练模型; 0: 使用负采样。  
1.skip-gram （训练速度慢，对罕见字有效），CBOW（训练速度快）。一般选择Skip-gram模型；  

2.训练方法：Hierarchical Softmax（对罕见字有利），Negative Sampling（对常见字和低维向量有利）；  

3.欠采样频繁词可以提高结果的准确性和速度（1e-3~1e-5）  

4.Window大小：Skip-gram通常选择10左右，CBOW通常选择5左右。  